In [8]:
from tensorflow import keras
model = keras.models.load_model('model_4.hdf5')


OSError: SavedModel file does not exist at: model.tflite/{saved_model.pbtxt|saved_model.pb}

In [2]:
import tensorflow as tf
import tensorflow.lite as lite
import tensorflow_datasets as tfds
dataset_name = "speech_commands"


speech_builder = tfds.builder(dataset_name)
for ele in tfds.list_builders():
    if "speech" in ele:
        print(ele)

dataset_name = "speech_commands"

# instantiate a dataset builder (see tensorflow dataset builder)
speech_builder = tfds.builder(dataset_name)
print(speech_builder.info)

# download data into existing data folder
speech_builder.download_and_prepare()
data = speech_builder.as_dataset(as_supervised=True)
test_set = data['test']
test_set_size = 4890


def make_mfccs(audio, labels):
    
    FRAME_RATE = 16000
    
    stfts = tf.signal.stft(tf.cast(audio, tf.float32), frame_length=1024, frame_step=256,
                       fft_length=1024)
    
    spectrograms = tf.abs(stfts)

    # Warp the linear scale spectrograms into the mel-scale.
    num_spectrogram_bins = stfts.shape[-1]
    lower_edge_hertz, upper_edge_hertz, num_mel_bins = 80.0, 7600.0, 80
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(num_mel_bins, num_spectrogram_bins,
                                                                        FRAME_RATE, lower_edge_hertz,
                                                                        upper_edge_hertz)
    
    mel_spectrograms = tf.tensordot(spectrograms, linear_to_mel_weight_matrix, 1)
    
    mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(linear_to_mel_weight_matrix.shape[-1:]))

    # Compute a stabilized log to get log-magnitude mel-scale spectrograms.
    log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

    # Compute MFCCs from log_mel_spectrograms and take the first 13.
    # You can use other parts, or even all of the MFCCs, to test around how it affect the accuracy
    mfcc = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrograms)[..., :13]
    
    # Finally add depth to the mfcc tensors, our Conv2D-Model requires this.
    mfcc = tf.expand_dims(mfcc, -1)
    
    return mfcc, labels

dataset_test = test_set.map(make_mfccs
                           ).padded_batch(128
                           ).prefetch(tf.data.experimental.AUTOTUNE)



gtzan_music_speech
librispeech
librispeech_lm
ljspeech
speech_commands
tfds.core.DatasetInfo(
    name='speech_commands',
    version=0.0.2,
    description='An audio dataset of spoken words designed to help train and evaluate keyword
spotting systems. Its primary goal is to provide a way to build and test small
models that detect when a single word is spoken, from a set of ten target words,
with as few false positives as possible from background noise or unrelated
speech. Note that in the train and validation set, the label "unknown" is much
more prevalent than the labels of the target words or background noise.
One difference from the release version is the handling of silent segments.
While in the test set the silence segments are regular 1 second files, in the
training they are provided as long segments under "background_noise" folder.
Here we split these background noise into 1 second clips, and also keep one of
the files for the validation set.',
    homepage='https://arxiv.org/a

In [13]:
def calc_acc(predictions, dataset):
    total, correct = 0, 0
    index = 0

    for batch in dataset:
        for b_ele in batch[1]:
            #print(b_ele)
            total += 1
            if b_ele == np.argmax(predictions[index]):
                correct += 1
            index += 1


    print("{} out of {} correct, resulting in an accuracy of {}\n".format(correct, total, (correct/total)*100))  

In [11]:
preds = model.predict(dataset_test, verbose=1)

39/39 [==============================] - 21s 547ms/step


In [14]:
calc_acc(preds, dataset_test)

4242 out of 4890 correct, resulting in an accuracy of 86.74846625766871



In [12]:
import numpy as np

total, correct = 0, 0
index = 0

for batch in dataset_test:
    for b_ele in batch[1]:
        #print(b_ele)
        total += 1
        if b_ele == np.argmax(preds[index]):
            correct += 1
        index += 1
        
        
print("{} out of {} correct, resulting in an accuracy of {}".format(correct, total, (correct/total)*100))    

4242 out of 4890 correct, resulting in an accuracy of 86.74846625766871


In [5]:
validation_set = data['validation']
validation_set_size = 10102
dataset_validation = validation_set.map(make_mfccs
                                       ).padded_batch(128
                                       ).prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
preds = model.predict(dataset_validation, verbose=1)

79/79 [==============================] - 76s 968ms/step


In [7]:
total, correct = 0, 0
index = 0

for batch in dataset_validation:
    for b_ele in batch[1]:
        #print(b_ele)
        total += 1
        if b_ele == np.argmax(preds[index]):
            correct += 1
        index += 1
        
        
print("{} out of {} correct, resulting in an accuracy of {}".format(correct, total, (correct/total)*100)) 

9493 out of 10102 correct, resulting in an accuracy of 93.9714907939022


In [10]:
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()


"""Need to convert tfds to numpy in oder to use it in numpy"""
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.0086206  0.006905   0.00485337 0.00393686 0.01810727 0.00993071
  0.00296913 0.00218761 0.01782713 0.00573496 0.0024187  0.9165087 ]]
